In [1]:
# # For google drive access
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import pandas as pd

# Data Prep

In [2]:
yarn_eng_needles = pd.read_csv("/notebooks/Vietnamese_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Indonesian_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Swahili_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Somali_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Azeri_needles.csv")


# --------------- 2-Needles ---------------
# yarn_eng_needles = pd.read_csv("/notebooks/Vietnamese_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Indonesian_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Swahili_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Somali_needles.csv")
# yarn_eng_needles = pd.read_csv("/notebooks/Azeri_needles.csv")

yarn_eng_needles.head(7)

,context_length,position,text,city1,label1,city2,label2
0,2k,0,Số Kuala Lumpur ma thuật đặc biệt là 4499006 ...,Kuala Lumpur,4499006,Yerevan,3659685
1,2k,25,"Nguồn hình ảnh, Getty ImagesCông ty của ông El...",Cairo,7108491,Yangon,6325464
2,2k,50,"Nguồn hình ảnh, Getty ImagesCông ty của ông El...",San Francisco,8680616,Bangalore,5356475
3,2k,75,"Nguồn hình ảnh, Getty ImagesCông ty của ông El...",Yerevan,6861767,Beirut,7970726
4,8k,0,Số Barcelona ma thuật đặc biệt là 7519349 Ngu...,Barcelona,7519349,Kampala,9708441
5,8k,25,"Nguồn hình ảnh, Win McNamee/Getty ImagesCựu T...",Paris,5717920,Lisbon,6071205
6,8k,50,"Nguồn hình ảnh, Win McNamee/Getty ImagesCựu T...",Bratislava,2819977,Mumbai,9847302


# Model - Yarn

In [4]:
# !nvcc --version
# !pip install torch==1.13
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1

import torch
print(torch.cuda.is_available())
print(torch.__version__)

True
2.2.1+cu121


In [1]:
# !pip3 install -q git+https://github.com/HazyResearch/flash-attention.git#subdirectory=csrc/rotary
!pip3 install -q git+https://github.com/huggingface/transformers.git
!pip3 install -q auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

!pip3 install -q git+https://github.com/huggingface/optimum.git
!pip3 install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.5 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 63.2 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.6-cp39-cp39-linux_x86_64.whl size=120593078 sha256=fe1699ff8cc792c0b9b86c1e428e733b12344e1409be5d34a8362f00cc22657c
  Stored in directory: /root/.cache/pip/wheels/68/e9/30/3cd6f67ab35cea745ea3300624d6ce0f41b5af93e3b50d3553
Successfully built flash-attn


In [3]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_id = "NousResearch/Yarn-Llama-2-13b-128k"
model_id = "NousResearch/Yarn-Llama-2-7b-128k"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             revision="main",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

>>>> Flash Attention installed
>>>> Flash RoPE installed


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Multiple GPUs

In [ ]:

import torch
from accelerate import Accelerator

accelerator = Accelerator()

generated_text = []

for i, text in enumerate(yarn_eng_needles['text'][16:]):
    ## 1-needle
#     input_context = f"{text}\\nHere is the magic number:"
    input_context = f'''
        You are a helpful AI bot that answers questions for a user. Keep your response short and direct.
        The following is a set of context and a question that will relate to the context. 
        #CONTEXT
        {text}
        #ENDCONTEXT

        #QUESTION
        What is the special magic {{}} number? Don't give information outside the document or repeat your findings. If the
        information is not available in the context respond UNANSWERABLE.
    '''

    ## 2-needles
        
    # prompt = f"{text}\nWhich city has the larger magic number?"
    # prompt = f'''
    # You are a helpful AI bot that answers questions for a user. Keep your response short and direct.
    # The following is a set of context and a question that will relate to the context. 
    # #CONTEXT
    # {text}
    # #ENDCONTEXT

    # #QUESTION
    # Which city has the larger magic {{}} number? Don't give information outside the document or repeat your findings. If the
    # information is not available in the context respond UNANSWERABLE.
    # '''
    
    input_ids = tokenizer.encode(input_context, return_tensors="pt")

    model, input_ids = accelerator.prepare(model, input_ids)

    with torch.no_grad():
        output = accelerator.unwrap_model(model).generate(input_ids,
                                max_new_tokens=50,
                                )


    output_text = tokenizer.decode(accelerator.gather(output)[0], skip_special_tokens=True)
    generated_text.append(output_text)

In [7]:
generated_text

[]

## Single GPU

### 1-Needle

In [ ]:
generated_text_1 = []

for i, text in enumerate(yarn_eng_needles['text']):
    # input_context = f"{text}\nHere is the magic number:"
    
    ## 2-needles
    # input_context = f"{text}\nCompare the magic number associated with the cities and give the name of the city with the larger magic number."
    
    # input_context = f"{text}\nThe above text has magic numbers for two cities. Give the name of the city with the larger magic number."
    # input_context = f"{text}\nWhich city has the larger magic number? Give one word answer."
    # input_context = f"{text}\nThe city with the larger magic number is:"
    
    input_context = f'''
    You are a helpful AI bot that answers questions for a user. Keep your response short and direct.
    The following is a set of context and a question that will relate to the context. 
    #CONTEXT
    {text}
    #ENDCONTEXT

    #QUESTION
    What is the special magic {{}} number? Don't give information outside the document or repeat your findings. If the
    information is not available in the context respond UNANSWERABLE.
    '''
    
    input_ids = tokenizer.encode(input_context, return_tensors="pt")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    model.to(device)

    with torch.no_grad():
        output = model.generate(input_ids,
                                max_new_tokens=50,
                                )

    output_text = tokenizer.decode(output[0], skip_special_tokens=True)

    generated_text_1.append(output_text)

In [5]:
len(generated_text_1)

16

In [7]:
df = pd.DataFrame(generated_text_1, columns=['generated_text'])

df.to_csv('yarn-7b_aze_eng_v1.csv', index=False)

### 2-Needles

In [ ]:
generated_text = []

for i, text in enumerate(yarn_eng_needles['text']):
    # input_context = f"{text}\nHere is the magic number:"
    
    ## 2-needles
    # input_context = f"{text}\nCompare the magic number associated with the cities and give the name of the city with the larger magic number."
    # input_context = f"{text}\nThe above text has magic numbers for two cities. Give the name of the city with the larger magic number."

    input_context = f"{text}\nWhich city has the larger magic number? Give one word answer."
    
    input_ids = tokenizer.encode(input_context, return_tensors="pt")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    model.to(device)

    with torch.no_grad():
        output = model.generate(input_ids,
                                max_new_tokens=50,
                                )

    output_text = tokenizer.decode(output[0], skip_special_tokens=True)

    generated_text.append(output_text)

In [7]:
len(generated_text)

12

In [8]:
df = pd.DataFrame(generated_text, columns=['generated_text'])

df.to_csv('yarn-7b-som_eng_v1_2n.csv', index=False)